In [ ]:
from read_docx_tables import read_docx_tables
import pandas as pd
import numpy as np

In [ ]:
semester = '22-23 s1/'

grades_from = ['1', '1-2', '2', '2-3', '3', '3-4', '4', '4-5', '5', '5-6', '6', 
                'disp', 'besucht', 'Fehlt']
grades_to = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, np.NaN, np.NaN, np.NaN]

mng_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ks_letters = ['p', 'q']

mng = [str(y) + l for y in range(1, 4) for l in mng_letters]
ksos = [str(y) + l for y in range(1, 5) for l in ks_letters]
ksus = ['u1', 'u2']

grades = {cl: read_docx_tables(semester+cl+'.docx')[1][:-1]
    .replace(grades_from, grades_to) for cl in mng}

for cl in ksos:
    data = read_docx_tables(semester+cl+'.docx')
    if (parts := len(data)//2) > 1:
        for i in range(parts):
            grades[f'{cl} {i+1}'] = data[2*i+1][:-1].replace(grades_from, grades_to)
    else:
        grades[cl] = data[1][:-1].replace(grades_from, grades_to)

grades |= {cl: read_docx_tables(semester+cl+'.docx')[1][:-1]
    .replace(grades_from, grades_to) for cl in ksus}


for df in grades.values():
    df.drop(columns = df.columns[df.columns.str.startswith('Unnamed')], inplace = True)

to_drop = ['Nr', 'Durchschnitt', 'Hpkt', 'Sum.Tpkt', 'Tnot', 'zahl', 'Entscheid', 'Entsch.', '\n']
subjects = ['D ', 'F ', 'I ', 'E ', 'M ', 'Alg ', 'Gm ', 'AM ', 'If ', 
            'B ', 'Ch ', 'P ', 'G ', 'Gg ', 'BG ', 'Mu ', 'Sp ', 'L ']
teachers = {}

for cl, df in grades.items():
    cols = df.columns.tolist()
    teachers[cl] = {}
    
    for i, cname in enumerate(cols):
        s = cname.split(' ')[0]
        s += ' '
        t = cname[2:]

        if s in subjects:
            if 'SP' in t:
                s = s.strip()+' SP'
            df.columns.values[i] = s.strip()
            teachers[cl][s.strip()] = [v.strip().split(' ', 1)[0] for v in t.split('/')]
        
    pos_drop = [i for i, c in enumerate(cols) for x in to_drop if x == c[-len(x):]]
    df.drop(df.columns[pos_drop], axis=1, inplace=True)
    
for df in grades.values():
    df.iloc[:, 2:].apply(pd.to_numeric)



In [ ]:
mng_drop = {'1': ['Alg', 'Gm', 'BG', 'Mu', 'Sp'],
            '2': ['If', 'Sp'],
            '3': ['Sp']}

ks_drop = {'1p': ['Alg', 'Gm'], '1q 1': ['Sp'], '1q 2': [],
            '2p': ['If'], '2q 1': ['If'], '2q 2': ['Sp', 'If'],
            '3p': [], '3q 1': [], '3q 2': 'Sp',
            '4p': [], '4q 1': ['Sp'], '4q 2': [],
            'u1': ['Sp'], 'u2': ['Sp']}

In [ ]:
listu1 = grades['u1'].drop(ks_drop['u1'], axis=1).mean(axis=0)
for s, a in zip(listu1.index, listu1):
    print(s, a)

In [ ]:
def average_by_subject(grades, subject):
    subj_sum = 0
    subj_cnt = 0

    for cdata in grades.values():
        if subject in cdata.columns:
            subj_sum += cdata[subject].sum()
            subj_cnt += cdata[subject].count()

    return subj_sum/subj_cnt


for subject in ['D', 'F', 'I', 'M', 'AM', 'AM SP', 
                'B', 'B SP', 'Ch', 'Ch SP', 'P', 'P SP', 'G', 'Gg', 'If', 'BG', 'Mu', 'BG&Mu ', 'Sp']:
    print(f'{subject}: {average_by_subject(grades, subject):.2f}')

In [ ]:
def average_by_subject_and_year(grades, subject, year):
    subj_sum = 0
    subj_cnt = 0

    for cname, cdata in grades.items():
        if str(year) == cname[0]:
            if subject in cdata.columns:
                subj_sum += cdata[subject].sum()
                subj_cnt += cdata[subject].count()

    if subj_cnt > 0:
        return subj_sum/subj_cnt
    else:
        return 0


for subject in ['D', 'F', 'I', 'M', 'AM', 'AM SP', 
                'B', 'B SP', 'Ch', 'Ch SP', 'P', 'P SP', 'G', 'Gg', 'If', 'BG', 'Mu', 'BG&Mu ', 'Sp']:
    av = [average_by_subject_and_year(grades, subject, year) for year in range(1, 4)]
    print(f'{subject}: {av[0]:.2f} {av[1]:.2f} {av[2]:.2f}')

In [ ]:
def average_per_teacher(grades, teachers, subject, teacher):
    teach_sum = 0
    teach_cnt = 0

    for cname, cdata in grades.items():
        if subject in teachers[cname]:
            if teacher in teachers[cname][subject]:
                teach_sum += cdata[subject].sum()
                teach_cnt += cdata[subject].count()
        
    if teach_cnt > 0:
        return teach_sum/teach_cnt
    else:
        return 0

for teacher in ['ByS', 'HaC', 'KeD', 'KrA', 'LiM', 'ThC', 'WeP']:
    print(f'{teacher}: {average_per_teacher(grades, teachers, "P SP", teacher):.2f}')

print(average_per_teacher(grades, teachers, 'Gg', 'KiR'))

In [ ]:
teachers['1a']